In [1]:
print("Starting with Deep Learning")

Starting with Deep Learning


In [2]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [3]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [4]:
print(tf.__version__)
print(keras.__version__)

2.6.0
2.6.0


In [5]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [6]:
### Usable Files

direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Filtered/Processed/")

# direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Unfiltered/Processed/")

file_list = list(direction.iterdir())



file_num = len(file_list)

num_test_files = 2
eighty = round(0.8*file_num)
twenty = file_num-eighty - num_test_files

list_of_file_ids_train = np.arange(eighty, dtype=int)
print(list_of_file_ids_train)
list_of_file_ids_val = np.arange(eighty,eighty+twenty-num_test_files, dtype=int)

list_of_file_ids_test =np.arange(file_num-num_test_files,file_num)



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]


In [7]:
##### Functions to process the data

        ### Unnormalization of each signal individually
def Unnormalized(batch_signals):
        
        return batch_signals
        
        ### Normalization of each signal individually
def Normalized(batch_signals):

        for i in range(len(batch_signals)):
            batch_signals[i] = batch_signals[i]/np.max(batch_signals[i])
            
        return batch_signals
            
        
        ### Normalization of the entire value by one common denominator      
def Denominator(batch_signals):  
    
        denominator = 3953.48
        batch_signals = batch_signals/denominator
        
        return batch_signals


##### Class

class TrainDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Train Class')
        if(file_id == 0):
#             print("reshuffling")
            np.random.shuffle(list_of_file_ids_train)             

        i_file = list_of_file_ids_train[file_id]

#         print(f'file_id: {file_id}, i_file: {i_file}')
#         print()
        signal_filename = direction/f'{i_file+1}.h5'

        
         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)  

        mask = (df['A'] == mass) & (df['Z'] == energy)
        df_new = df[~mask]       

        
        labels1 = df_new.iloc[:,9].values
        labels2 = df_new['ToF'].values
        labels = labels1+labels2
        
        signals = df_new[df_new.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels 
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )
    
class ValDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Val Class')
        i_file = list_of_file_ids_val[file_id]
    
        signal_filename = direction/f'{i_file+1}.h5'

         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)  

        mask = (df['A'] == mass) & (df['Z'] == energy)
        df_new = df[~mask]       

        
        labels1 = df_new.iloc[:,9].values
        labels2 = df_new['ToF'].values
        labels = labels1+labels2
        
        signals = df_new[df_new.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )

In [8]:
df = pd.read_hdf(file_list[0],key=None)  

labels1 = df.iloc[:,9].values
labels2 = df['ToF'].values
labels = labels1+labels2

print(labels1)

[499.14654541 499.83734131 499.82849121 ... 499.1619873  499.05609131
 499.6555481 ]


In [9]:
# model = keras.models.Sequential()
# model.add(Conv1D(filters=16, kernel_size=5,activation='relu', input_shape=(1998, 1)))
# model.add(Conv1D(filters=8, kernel_size=5,dilation_rate=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=5,dilation_rate=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=5,strides=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=3,strides=2))
# model.add(Flatten())
# model.add(Dense(32,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dense(1))

# print(model.summary())


In [10]:
def create_model():
    model = keras.models.Sequential()
    model.add(Conv1D(filters=16, kernel_size=5, activation='relu', input_shape=(1998, 1)))
    model.add(Conv1D(filters=8, kernel_size=5, dilation_rate=2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=4, kernel_size=5, dilation_rate=2, activation='relu'))
    model.add(Conv1D(filters=4, kernel_size=5, strides=2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=4, kernel_size=3, strides=2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    return model

In [11]:
# Pre-processing options
Processing = {
    "Unnormalized": Unnormalized,
    "Normalized": Normalized,
    "Denominator": Denominator
}
Process = ["Unnormalized","Normalized","Denominator"]
process = Process[2]
# Loss Function

loss_function = ['mean_absolute_error','mean_squared_error']
lf = 1

# Training Variables
batch_size = 32
num_epochs = 30

steps_per_epoch = eighty*5000 // batch_size

# Learning Rate
initial_lr = 1e-03
final_lr = 1e-06

# initial_lr = 1e-03
# final_lr = 1e-03

def step_decay(epoch):
    lrate = initial_lr * (final_lr/initial_lr)**(epoch/num_epochs)

    print(f'Current Learning rate: {lrate}')
    return lrate

# Compile the model
# model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

# Configuring training dataset
dataset_train = tf.data.Dataset.range(eighty).interleave(
        TrainDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).repeat().prefetch(1)



# Configuring training dataset
dataset_val = tf.data.Dataset.range(twenty-num_test_files).interleave(
        ValDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).prefetch(1)


# Callback Functions
LRS = tf.keras.callbacks.LearningRateScheduler(step_decay)

ES = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True,verbose=1)

CSV = tf.keras.callbacks.CSVLogger("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Log.csv",
                                separator=",", append=True)

MC_path = f"C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/model_checkpoint.h5"
MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
)

callbacks = [MC,LRS,CSV]


Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [12]:
# Define the path to the files
MC_path = "C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/Cont"
process = Process[2]


# Get the list of model file names
file_models = os.listdir(MC_path)

print(file_models)

# Step 1: Group by 'A' (mass) and 'Z' (energy) and count the occurrences
grouped = df.groupby(['A', 'Z']).size().reset_index(name='count')

# pd.read_hdf(file, key=None)

['model_100.0_108.0_checkpoint_1.h5', 'model_100.0_108.0_checkpoint_2.h5', 'model_100.0_108.0_checkpoint_3.h5', 'model_100.0_34.0_checkpoint_1.h5', 'model_100.0_34.0_checkpoint_2.h5', 'model_100.0_34.0_checkpoint_3.h5', 'model_100.0_41.0_checkpoint_1.h5', 'model_100.0_41.0_checkpoint_2.h5', 'model_100.0_41.0_checkpoint_3.h5', 'model_100.0_48.0_checkpoint_1.h5', 'model_100.0_48.0_checkpoint_2.h5', 'model_100.0_48.0_checkpoint_3.h5', 'model_100.0_58.0_checkpoint_1.h5', 'model_100.0_58.0_checkpoint_2.h5', 'model_100.0_58.0_checkpoint_3.h5', 'model_100.0_68.0_checkpoint_1.h5', 'model_100.0_68.0_checkpoint_2.h5', 'model_100.0_68.0_checkpoint_3.h5', 'model_100.0_78.0_checkpoint_1.h5', 'model_100.0_78.0_checkpoint_2.h5', 'model_100.0_78.0_checkpoint_3.h5', 'model_100.0_88.0_checkpoint_1.h5', 'model_100.0_88.0_checkpoint_2.h5', 'model_100.0_88.0_checkpoint_3.h5', 'model_100.0_98.0_checkpoint_1.h5', 'model_100.0_98.0_checkpoint_2.h5', 'model_100.0_98.0_checkpoint_3.h5', 'model_104.0_22.0_checkp

In [13]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import pathlib
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define the path to the files
MC_path = "C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/Cont"
process = Process[2]  # This is required for specifying which preprocessing you use

# Get the list of model file names
file_models = os.listdir(MC_path)

# Create dictionaries to store metrics for each combination of mass and energy
mae_dict = {}
mse_dict = {}
mean_diff_dict = {}
std_diff_dict = {}

# Loop over all files
for model_file in file_models:
    # Extract the mass and energy values from the model file name
    mass, energy = float(model_file.split("_")[1]), float(model_file.split("_")[2])

    # Load the data from the corresponding file
    # Assuming you want to load the first data file in the list (file_list[0])
    for file in file_list:
        # Change this index according to your requirement
        df = pd.read_hdf(file, key=None)

        # Assuming you have a function 'create_model()' that returns the compiled model
        model = create_model()

        # Load the pre-trained model weights from the file
        model.load_weights(os.path.join(MC_path, model_file))

        # Continue with the rest of the code to make predictions and perform statistics
        mask = (df['A'] == mass) & (df['Z'] == energy)
        df_new = df[mask]
        print(len(df_new))
        # Assuming you have a pre-trained model loaded, let's make predictions and calculate the differences between predicted and true labels.
        labels1 = df_new.iloc[:, 9].values
        labels2 = df_new['ToF'].values
        labels = labels1 + labels2

        signals = df_new[df_new.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels
        print(len(predicted-true))

        if len(predicted-true) == 1:
            # Handle the case where both 'predicted' and 'true' are scalars
            # Calculate a custom error metric, such as absolute difference
            custom_error = abs(predicted - true)
            print("Custom Error (Absolute Difference):", custom_error)

            # You can add additional handling or metrics specific to scalar values here

            # Include the custom error in the overall metrics dictionaries
            key = (mass, energy)
            mae_dict.setdefault(key, []).append(custom_error)
            mse_dict.setdefault(key, []).append(custom_error)
            mean_diff_dict.setdefault(key, []).append(custom_error)
            std_diff_dict.setdefault(key, []).append(custom_error)
        else:
            # Calculate metrics for arrays of values
            mae = mean_absolute_error(true, predicted)
            mse = mean_squared_error(true, predicted)
            mean_diff = np.mean(predicted - true)
            std_diff = np.std(predicted - true)

            # Store metrics in dictionaries based on mass and energy values
            key = (mass, energy)
            mae_dict.setdefault(key, []).append(mae)
            mse_dict.setdefault(key, []).append(mse)
            mean_diff_dict.setdefault(key, []).append(mean_diff)
            std_diff_dict.setdefault(key, []).append(std_diff)

# Calculate mean and standard deviation of the metrics for each group (combination of mass and energy)
combined_metrics = {}
for key in mae_dict.keys():
    mae_mean = np.mean(mae_dict[key])
    mae_std = np.std(mae_dict[key])
    mse_mean = np.mean(mse_dict[key])
    mse_std = np.std(mse_dict[key])
    mean_diff_mean = np.mean(mean_diff_dict[key])
    mean_diff_std = np.std(mean_diff_dict[key])
    std_diff_mean = np.mean(std_diff_dict[key])
    std_diff_std = np.std(std_diff_dict[key])

    combined_metrics[key] = {
        "MAE": (mae_mean, mae_std),
        "MSE": (mse_mean, mse_std),
        "Mean Difference": (mean_diff_mean, mean_diff_std),
        "Std Difference": (std_diff_mean, std_diff_std)
    }

# Print or use the calculated metrics as required
for key in combined_metrics:
    print(f"Mass={key[0]}, Energy={key[1]}")
    print("MAE: Mean =", combined_metrics[key]["MAE"][0], "Std =", combined_metrics[key]["MAE"][1])
    print("MSE: Mean =", combined_metrics[key]["MSE"][0], "Std =", combined_metrics[key]["MSE"][1])
    print("Mean Difference: Mean =", combined_metrics[key]["Mean Difference"][0], "Std =", combined_metrics[key]["Mean Difference"][1])
    print("Std Difference: Mean =", combined_metrics[key]["Std Difference"][0], "Std =", combined_metrics[key]["Std Difference"][1])

17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
17
17
16
16
17
17
17
17
17
17
17
17
104
104
104
104
104
104
104
104
104
104
104
104
105
105
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
105
105
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
105
105
104
104
104
104
104
104
104
104
104
104
104
104


86
86
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
170
170
169
169
169
169
170
170
170
170
169
169
170
170
169
169
170
170
59
59
60
60
59
59
59
59
60
60
59
5

381
381
382
382
382
382
382
382
382
382
381
381
382
382
382
382
381
381
381
381
382
382
382
382
382
382
382
382
381
381
382
382
382
382
382
382
381
381
382
382
382
382
382
382
382
382
381
381
382
382
382
382
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
289
288
288
289
289
289
289
289
289
289
289
289
289
289
289


199
199
200
200
200
200
199
199
200
200
200
200
199
199
200
200
199
199
200
200
199
199
200
200
199
199
199
199
200
200
200
200
199
199
200
200
199
199
200
200
200
200
200
200
199
199
200
200
199
199
200
200
199
199
200
200
199
199
200
200
199
199
199
199
200
200
200
200
199
199
200
200
199
199
200
200
199
199
200
200
199
199
200
200
200
200
199
199
200
200
200
200
199
199
200
200
199
199
200
200
199
199
200
200
199
199
199
199
200
200
200
200
199
199
200
200
199
199
200
200
333
333
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
332
332
333
333
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
333
333
333
333
332
332
333
333
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
332
332
333
333
333
333
332
332
333
333
332
332
333
333
333
333
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
332
332
333
333
333
333
332
332
333
333
332
332
333
333
332
332
333
333
332
332
333
333
333
333
332
332
333
333


197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
198
198
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
197
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
58
58
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
58
58
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
58
58
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
58
58
57
57
57
57
57
57
57
57
57
57
57
57
58
58
57
57
113
113
113
113
113
113
114
114
113
113
113
113
113
113
113
113
113
113
113
113


101
101
100
100
101
101
101
101
101
101
100
100
101
101
101
101
100
100
101
101
101
101
101
101
100
100
101
101
101
101
101
101
100
100
101
101
101
101
100
100
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
526
526
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
525
525
526
526
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
526
526
525
525
525
525
526
526
525
525
526
526
525
525
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
526
526
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
525
525
526
526
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
526
526
525
525
525
525
526
526
525
525
526
526
525
525
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
526
526
525
525
525
525
526
526
525
525
526
526
525
525
526
526
525
525
526
526
525
525
525
525
526
526
525
525


1
1
Custom Error (Absolute Difference): [0.03063131]
1
1
Custom Error (Absolute Difference): [0.16342997]
2
2
1
1
Custom Error (Absolute Difference): [0.03861307]
1
1
Custom Error (Absolute Difference): [0.09245993]
1
1
Custom Error (Absolute Difference): [0.12183451]
1
1
Custom Error (Absolute Difference): [0.03272318]
2
2
1
1
Custom Error (Absolute Difference): [0.17292093]
1
1
Custom Error (Absolute Difference): [0.13266825]
2
2
2
2
1
1
Custom Error (Absolute Difference): [0.0226829]
1
1
Custom Error (Absolute Difference): [0.02515482]
1
1
Custom Error (Absolute Difference): [0.14630961]
2
2
1
1
Custom Error (Absolute Difference): [0.03836893]
1
1
Custom Error (Absolute Difference): [0.16147684]
1
1
Custom Error (Absolute Difference): [0.11994242]
2
2
1
1
Custom Error (Absolute Difference): [0.0902183]
1
1
Custom Error (Absolute Difference): [0.21732401]
1
1
Custom Error (Absolute Difference): [0.06235575]
1
1
Custom Error (Absolute Difference): [0.02414774]
2
2
1
1
Custom Error (Ab

16
16
15
15
16
16
16
16
16
16
15
15
15
15
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
15
15
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
15
15
16
16
16
16
16
16
16
16
15
15
16
16
16
16
16
16
15
15
16
16
16
16
16
16
16
16
16
16
15
15
16
16
16
16
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
61
61
61
60
60
61
61
61
61
61
61
61
61
61
61
61
6

273
273
273
273
273
273
274
274
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
273
273
274
274
273
273
273
273
274
274
195
195
194
194
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
194
194
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
195
195
195
195
194
194
195
195
195
195
194
194
195
195
195
195
195
195
194
194
195
195
195
195
194
194


C:\Users\steve\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
C:\Users\steve\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:198: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


In [14]:
# Calculate mean and standard deviation of the metrics for each group (combination of mass and energy)
combined_metrics = {}
for key in mae_dict.keys():
    mae_mean = np.mean(mae_dict[key])
    mae_std = np.std(mae_dict[key])
    mse_mean = np.mean(mse_dict[key])
    mse_std = np.std(mse_dict[key])
    mean_diff_mean = np.mean(mean_diff_dict[key])
    mean_diff_std = np.std(mean_diff_dict[key])
    std_diff_mean = np.mean(std_diff_dict[key])
    std_diff_std = np.std(std_diff_dict[key])

    combined_metrics[key] = {
        "MAE": (mae_mean, mae_std),
        "MSE": (mse_mean, mse_std),
        "Mean Difference": (mean_diff_mean, mean_diff_std),
        "Std Difference": (std_diff_mean, std_diff_std)
    }

# Print or use the calculated metrics as required
for key in combined_metrics:
    print(f"Mass={key[0]}, Energy={key[1]}")
    print("MAE: Mean =", combined_metrics[key]["MAE"][0], "Std =", combined_metrics[key]["MAE"][1])
    print("MSE: Mean =", combined_metrics[key]["MSE"][0], "Std =", combined_metrics[key]["MSE"][1])
    print("Mean Difference: Mean =", combined_metrics[key]["Mean Difference"][0], "Std =", combined_metrics[key]["Mean Difference"][1])
    print("Std Difference: Mean =", combined_metrics[key]["Std Difference"][0], "Std =", combined_metrics[key]["Std Difference"][1])

Mass=100.0, Energy=108.0
MAE: Mean = 0.12680292738841684 Std = 0.036201851916312595
MSE: Mean = 0.02651997767268947 Std = 0.013309183636781642
Mean Difference: Mean = 0.07338214038123295 Std = 0.04941013367200623
Std Difference: Mean = 0.13035722280522502 Std = 0.041239208270551095
Mass=100.0, Energy=34.0
MAE: Mean = 0.09429923559853329 Std = 0.010510056355058054
MSE: Mean = 0.01738668750473577 Std = 0.005570038789149688
Mean Difference: Mean = 0.015314029693662353 Std = 0.04749074553687992
Std Difference: Mean = 0.12007108209850338 Std = 0.021902792774511178
Mass=100.0, Energy=41.0
MAE: Mean = 0.09162542305656755 Std = 0.009844636768361458
MSE: Mean = 0.015934317144595648 Std = 0.005531795155038048
Mean Difference: Mean = 0.06595345567251786 Std = 0.012330067773053248
Std Difference: Mean = 0.10523611137520884 Std = 0.018915314451137893
Mass=100.0, Energy=48.0
MAE: Mean = 0.08491131204518981 Std = 0.011958741211599901
MSE: Mean = 0.012283860875876984 Std = 0.00373900049731029
Mean Dif

In [15]:
# Calculate mean and standard deviation of the metrics across all groups
all_mae_values = [value for values in mae_dict.values() for value in values]
all_mse_values = [value for values in mse_dict.values() for value in values]
all_mean_diff_values = [value for values in mean_diff_dict.values() for value in values]
all_std_diff_values = [value for values in std_diff_dict.values() for value in values]

mae_mean_all = np.mean(all_mae_values)
mae_std_all = np.std(all_mae_values)
mse_mean_all = np.mean(all_mse_values)
mse_std_all = np.std(all_mse_values)
mean_diff_mean_all = np.mean(all_mean_diff_values)
mean_diff_std_all = np.std(all_mean_diff_values)
std_diff_mean_all = np.mean(all_std_diff_values)
std_diff_std_all = np.std(all_std_diff_values)

# Create a dictionary for the combined metrics
combined_metrics_all = {
    "MAE": (mae_mean_all, mae_std_all),
    "MSE": (mse_mean_all, mse_std_all),
    "Mean Difference": (mean_diff_mean_all, mean_diff_std_all),
    "Std Difference": (std_diff_mean_all, std_diff_std_all)
}

# Print or use the combined metrics as required
print("Combined Metrics for All Mass and Energy:")
print("MAE: Mean =", combined_metrics_all["MAE"][0], "Std =", combined_metrics_all["MAE"][1])
print("MSE: Mean =", combined_metrics_all["MSE"][0], "Std =", combined_metrics_all["MSE"][1])
print("Mean Difference: Mean =", combined_metrics_all["Mean Difference"][0], "Std =", combined_metrics_all["Mean Difference"][1])
print("Std Difference: Mean =", combined_metrics_all["Std Difference"][0], "Std =", combined_metrics_all["Std Difference"][1])

Combined Metrics for All Mass and Energy:
MAE: Mean = [0.09777994] Std = [0.07482651]
MSE: Mean = [0.02456788] Std = [0.08567104]
Mean Difference: Mean = [0.00516768] Std = [0.09497746]
Std Difference: Mean = [0.11010989] Std = [0.05265729]


In [16]:
# Create a dictionary for the combined metrics
combined_metrics_all = {
    "MAE": (mae_mean_all, mae_std_all),
    "MSE": (mse_mean_all, mse_std_all),
    "Mean Difference": (mean_diff_mean_all, mean_diff_std_all),
    "Std Difference": (std_diff_mean_all, std_diff_std_all)
}

# Print or use the combined metrics as required
print("Combined Metrics for All Mass and Energy:")
for metric_name, (mean_value, std_value) in combined_metrics_all.items():
    mean_value = float(mean_value)
    std_value = float(std_value)
    print(f"{metric_name}: {mean_value:.3f} \u00B1 {std_value:.3f}")

Combined Metrics for All Mass and Energy:
MAE: 0.098 ± 0.075
MSE: 0.025 ± 0.086
Mean Difference: 0.005 ± 0.095
Std Difference: 0.110 ± 0.053


In [18]:
# Initialize lists to store all data points for each metric
all_mae_values = []
all_mse_values = []
all_mean_diff_values = []
all_std_diff_values = []

# Collect all data points for each metric
for key in mae_dict.keys():
    all_mae_values.extend(mae_dict[key])
    all_mse_values.extend(mse_dict[key])
    all_mean_diff_values.extend(mean_diff_dict[key])
    all_std_diff_values.extend(std_diff_dict[key])

# Calculate the overall mean and standard deviation for each metric
overall_mae_mean = np.mean(all_mae_values)
overall_mae_std = np.std(all_mae_values)
overall_mse_mean = np.mean(all_mse_values)
overall_mse_std = np.std(all_mse_values)
overall_mean_diff_mean = np.mean(all_mean_diff_values)
overall_mean_diff_std = np.std(all_mean_diff_values)
overall_std_diff_mean = np.mean(all_std_diff_values)
overall_std_diff_std = np.std(all_std_diff_values)

# Print or use the overall metrics as required
print("Overall MAE: Mean =", overall_mae_mean, "Std =", overall_mae_std)
print("Overall MSE: Mean =", overall_mse_mean, "Std =", overall_mse_std)
print("Overall Mean Difference: Mean =", overall_mean_diff_mean, "Std =", overall_mean_diff_std)
print("Overall Std Difference: Mean =", overall_std_diff_mean, "Std =", overall_std_diff_std)

Overall MAE: Mean = [0.09777994] Std = [0.07482651]
Overall MSE: Mean = [0.02456788] Std = [0.08567104]
Overall Mean Difference: Mean = [0.00516768] Std = [0.09497746]
Overall Std Difference: Mean = [0.11010989] Std = [0.05265729]
